In [ ]:
# Imports
from ml_utils import *
import os
os.environ['TF_CPP_MIN_LOG_LEVEL'] = '0' #Don't print TF INFO messages

import ipywidgets as widgets
import matplotlib as mpl
from ipywidgets import interact
from pprint import pprint
import datetime
import os
import pandas as pd
import seaborn as sns
import matplotlib.pyplot as plt
import numpy as np
import scipy
from sklearn.metrics import classification_report, confusion_matrix, ConfusionMatrixDisplay, f1_score
from tqdm.notebook import tqdm
from sklearn.model_selection import train_test_split
import wandb
import yaml

from tensorflow.keras import layers
import tensorflow as tf
from tensorflow import keras
keras.utils.set_random_seed(42)
# Set the font to computer modern
mpl.rc('font', family='serif', serif='cmr10')
mpl.rc('axes.formatter', use_mathtext=True)

%load_ext autoreload
%autoreload 2

In [ ]:
# Read in the existing config, optionally update it, and then save the result to disk again
with open('config.yaml', 'r') as file:
    config = yaml.safe_load(file)

config.update({
    'batch_size': 128,
    'center_label': True,
    'label_expansion': 8,
    'window_size': 14,
    'epochs': 500,
    'n_hidden_units': {1: 175, 2: 97},
    'architecture': 'ffnn',
    'dropout_frac': 0.56,
    'use_class_weights': False,
    'bias_final_layer': True,
    'lr': 0.0001832,
})

with open('config.yaml', 'w') as file:
    yaml.dump(config, file)

pprint(config)

TODO: you're not using softmax even though you mention it in the report. Also the biases need to be recalculated because softmax is no longer being used

In [ ]:
df = parse_csvs()

config, g2i, i2g, i2ohe, ohe2i, X, y, X_train, X_valid, y_train, y_valid = \
    build_dataset(df, config)
    
# to_print = [
#     (i2g(i), c, round(config['class_weight'][i], 6)) 
#     for i, c in 
#     zip(*np.unique(y, return_counts=True))
# ]
# print('\n'.join([f'{g: <12} {c: >5} {f: >7}' for g, c, f in to_print]))


In [ ]:
# Tensorflow Feed Forward NN
wandb.init()
history, model = compile_and_fit(
    X_train,
    y_train,
    X_valid,
    y_valid,
    config | {'epochs': 15},
    i2g,
    verbose=1,
)

In [ ]:
print('Percentage g255: ', (y_train == g2i('gesture0255')).sum() / y_train.shape)
eval_and_save(
    model, 
    X_train, 
    y_train, 
    X_valid, 
    y_valid, 
    config,
    i2g,
    history, 
    show_figs=True, 
    cbar=True, 
    make_plots=True,
    perc='both',
)

In [ ]:
to_predict, _count = sorted(list(zip(*np.unique(y_valid, return_counts=True))), key=lambda x: -x[1])[0]

y_pred = tf.one_hot(np.full(y_valid.shape, to_predict), len(np.unique(y_valid)))
dumb_scce = keras.losses.sparse_categorical_crossentropy(y_valid, y_pred).numpy()
dumb_scce.mean()

In [ ]:
model.summary()

#### Use DTW distance metric

In [ ]:
y_pred = model.predict(X, verbose=0)

In [ ]:
from pyts.metrics import dtw

In [ ]:
window_size = 0.1
dtw, path = dtw(
    x, y, dist='square', method='sakoechiba',
    options={'window_size': window_size}, return_path=True
)

In [ ]:
y_true = y_valid
y_pred = tf.one_hot(
    np.full(y_true.shape, g2i('gesture0255')), 
    len(np.unique(y_valid))
)
dumb_scce = keras.losses.sparse_categorical_crossentropy(
    y_true,
    y_pred,
).numpy().mean()
p, r, f1, sup = classification_report(y_true, np.argmax(y_pred, axis=-1), output_dict=True)['50'].values()

In [ ]:
@interact(start=widgets.IntSlider(
    value=275,
    min=0,
    max=y_pred.shape[0],
    step=100,
    description='Start',
    continuous_update=False,
), duration=widgets.IntSlider(
    value=20,
    min=0,
    max=5000,
    step=20,
    continuous_update=False,
    description='Duration',
))
def plt_preds(start, duration):
    s, f = start, start + duration
    fig, axs = plt.subplots(3, 1, figsize=(10, 5))
#     axs.formatter.use_mathtext = True
    onehot_valid = tf.one_hot(y[s:f], y_pred.shape[1]).numpy()

    axs[0].set_title('Actual gesture probabilities')
    sns.heatmap(
        onehot_valid.T,
        vmax=1,
        vmin=0,
        ax=axs[0]
    )

    axs[1].set_title('Predicted gesture probabilities')
    sns.heatmap(
        y_pred[s:f].T,
        ax=axs[1],
        vmax=1,
        vmin=0,

    )

    axs[2].set_title('Actual gesture probabilities minus predicted gesture probabilities')
    sns.heatmap(
        onehot_valid.T - y_pred[s:f].T,
        ax=axs[2],
        cmap='icefire',
        vmax=1,
        vmin=-1,

    )
    for ax in axs:
        ax.set_xticks([]) 
        ax.set_yticks([]) 
        ax.set_ylabel('Gesture')
    axs[-1].set_xlabel('Time')

    # plt.suptitle('Predicted and actual gestures over time')
    plt.tight_layout()


### Make predictions and create a simplified confusion matrix

In [ ]:
y_pred_valid = np.argmax(model.predict(X_valid, verbose=0), axis=1)

In [ ]:
confusion_mtx = tf.math.confusion_matrix(y_valid, y_pred_valid).numpy()

df = pd.DataFrame()
df.index = i2g(list(range(len(confusion_mtx[0]))))
total_obs = pd.Series(confusion_mtx.sum(axis=1), index=df.index)
df['pred=true'] = np.diag(confusion_mtx) / total_obs
df['pred=g255'] = confusion_mtx[:, -1] / total_obs
df['pred!=true!=g255'] = (1 - df['pred=true'] - df['pred=g255'])

df['pred=other true=g255'] = confusion_mtx[-1, :] / total_obs[-1]
df
# total_obs


fig, ax = plt.subplots(1, 1, figsize=(5, 10))
sns.heatmap(
    df.iloc[:-1, :],
    xticklabels=['%Correct', '% classified as g255', '%Some other gesture', '%actually g255, predicted other'],
    ax=ax,
    annot=True,
    fmt='.4f',
    cbar=False,
)
plt.title('Percentage of observations classified correctly, as g255,\nor as some other gesture')

### Visualise predictions

In [ ]:
# Visualise the predictions
df = parse_csvs()
config, g2i, i2g, i2ohe, ohe2i, X, y, X_train, X_valid, y_train, y_valid = \
    build_dataset(df, config)

window_size = config['window_size']
@interact(idx=(2*window_size, len(df) - window_size, 5))
def view_predictions(idx=window_size):
    if idx < 2*window_size or idx > len(df) - window_size:
        print(f"Clamping idx to between {2*window_size} and {len(df) - window_size}")
        idx = min(len(df) - window_size, max(idx, 2*window_size))
    
    y_orig = df['gesture'].to_numpy()
    X_orig = df.drop(['datetime', 'gesture'], axis=1).to_numpy()
    t_orig = df['datetime'].to_numpy()

    s, f = idx-window_size, idx+1#+window_size
    X_window = X[s:f]
    y_window = y[s:f]
    

    shape = model.get_config()['layers'][0]['config']['batch_input_shape']
    assert shape[1] == X_window.shape[1] and shape[2] == X_window.shape[2], \
        f'Shape in config is not the shape of the model'
    proba_preds = model.predict(X_window, verbose=0)
    X_window = X_window[:, 0, :]
    
    mask = np.max(proba_preds, axis=1) < 0.0
    preds = i2g(np.argmax(proba_preds, axis=1))
    preds[mask] = 'gesture0255'
    preds = [g.replace('gesture0', 'g').replace('g255', '') for g in preds]

#     plot_timeseries(
#         X_window,
#         i2g(y_window),
#         preds,
#         per='finger'
#     )
#     plt.suptitle(f'Observation at {idx}')
#     plt.show()
#     # TODO also show prediction probabilities over time

    fig, axs = plt.subplots(2, 1, figsize=(6, 7))
    sns.heatmap(
        X[idx].T,
        xticklabels=[g.replace('gesture0', 'g').replace('g255', '') for g in i2g(y_window)],
        ax=axs[0],
        cbar=None,
        vmax=900,
        vmin=300,
    )
#     axs[0].set_xticklabels(
#         [g.replace('gesture0', 'g').replace('g255', '') for g in i2g(y_window)],
#         rotation=90
#     )
    axs[0].set_title(f'Heatmap of Sensor measurements')
    axs[0].set_xlabel(f'Actual labels')
    axs[0].set_ylabel(f'Sensor')
    cm = plt.get_cmap('tab20')
    NUM_COLORS = len(np.unique(y_orig))

    axs[1].set_prop_cycle(color=([cm(1.*i/NUM_COLORS) for i in range(NUM_COLORS)]))
    axs[1].plot(
        proba_preds, 
        label=np.unique(y_orig),
    )
    axs[1].set_title(f'Lineplot of model predictions')
    axs[1].set_xticks(range(X[idx].shape[0]))
    axs[1].set_ylim((0, 1))
    axs[1].set_xlabel(f'Predicted Labels')
    axs[1].set_ylabel(f'Softmax of Final Layer')
    axs[1].set_xticklabels(preds[-X[idx].shape[0]:], rotation=90)
    plt.tight_layout()
    plt.show()
print("TODO: Make this show a larger region at once")

In [ ]:
df = parse_csvs()
config, g2i, i2g, i2ohe, ohe2i, X, y, X_train, X_valid, y_train, y_valid = \
    build_dataset(df, config | {'label_expansion': 0})
idx = np.where(y == g2i('gesture0036'))[0][34]
s = idx-16 - 10
f = idx+16 - 10
labels = [str(dt).replace('2022-10-18 ', '')[:-3] for dt in df.iloc[s:f]['datetime']]
vals = X[s:f, -1, :]
normed = (vals - vals.mean(axis=0)) / vals.std(axis=0)

plot_timeseries(normed, i2g(y[s:f]), labels, per='finger')
plt.tight_layout()
plt.savefig('../../report/imgs/gesture_over_time.pdf')

### Make figure for describing the training procedure

In [ ]:
df = parse_csvs()
config, g2i, i2g, i2ohe, ohe2i, X, y, X_train, X_valid, y_train, y_valid = \
    build_dataset(df, config | {'label_expansion': 0})
idx = np.where(y == g2i('gesture0001'))[0][19]
s = idx-50 - 6
f = idx+15 - 6
# labels = [str(dt).replace('2022-10-08 ', '')[:-3] for dt in df.iloc[s:f]['datetime']]
labels = [f'timestep {i:0>2}' for i in range(len(df.iloc[s:f]['datetime']))]
vals = X[s:f, -1, :]
# normed = (vals - vals.mean(axis=0)) / vals.std(axis=0)

plot_timeseries(X=vals, y=i2g(y[s:f]), t=labels, per='finger')
plt.tight_layout()
plt.savefig('../../report/imgs/explain_recording_procedure_raw.pdf')

## Weights And Biases Hyperparameter Sweeps

In [ ]:

with open('sweep.yaml', "r") as file:
    sweep_config = yaml.safe_load(file)
sweep_id = wandb.sweep(sweep_config, project="ergo")